## **Prepocessing**

In [ ]:
import pandas as pd
train_data = pd.read_excel('/content/drive/MyDrive/SPH 6004/Assignment 2/Version 3/Timestep5_train.xlsx')
test_data = pd.read_excel('/content/drive/MyDrive/SPH 6004/Assignment 2/Version 3/Timestep5_test.xlsx')
holdout_data = pd.read_excel('/content/drive/MyDrive/SPH 6004/Assignment 2/Version 3/Timestep5_holdout.xlsx')

In [ ]:
train_data_cat0 = train_data[train_data['icu_cat'] == 0]
train_data_cat1 = train_data[train_data['icu_cat'] == 1]

test_data_cat0 = test_data[test_data['icu_cat'] == 0]
test_data_cat1 = test_data[test_data['icu_cat'] == 1]

holdout_data_cat0 = holdout_data[holdout_data['icu_cat'] == 0]
holdout_data_cat1 = holdout_data[holdout_data['icu_cat'] == 1]

In [ ]:
from sklearn.preprocessing import StandardScaler
import numpy as np

numeric_columns_cat0 = train_data_cat0.select_dtypes(include=[np.number]).columns.tolist()
numeric_columns_cat1 = train_data_cat1.select_dtypes(include=[np.number]).columns.tolist()

train_numeric_cat0 = train_data_cat0[numeric_columns_cat0].drop(columns=['los_icu', 'icu_death','icu_cat'])
test_numeric_cat0 = test_data_cat0[numeric_columns_cat0].drop(columns=['los_icu', 'icu_death','icu_cat'])
holdout_numeric_cat0 = holdout_data_cat0[numeric_columns_cat0].drop(columns=['los_icu', 'icu_death','icu_cat'])

train_numeric_cat1 = train_data_cat1[numeric_columns_cat1].drop(columns=['los_icu', 'icu_death','icu_cat'])
test_numeric_cat1 = test_data_cat1[numeric_columns_cat1].drop(columns=['los_icu', 'icu_death','icu_cat'])
holdout_numeric_cat1 = holdout_data_cat1[numeric_columns_cat1].drop(columns=['los_icu', 'icu_death','icu_cat'])

print(train_numeric_cat1.columns)

Index(['id', 'aniongap', 'bicarbonate', 'bun', 'calcium', 'chloride',
       'creatinine', 'glucose', 'sodium', 'potassium', 'hematocrit',
       'hemoglobin', 'mch', 'mchc', 'mcv', 'platelet', 'rbc', 'rdw', 'wbc',
       'inr', 'pt', 'ptt', 'gender', 'admission_age', 'weight_admit', 'height',
       'charlson_score', 'atrial_fibrillation', 'malignant_cancer', 'chf',
       'ckd', 'cld', 'copd', 'diabetes', 'hypertension', 'ihd', 'stroke',
       'race_encode_African', 'race_encode_Asian', 'race_encode_Caucasian',
       'race_encode_Hispanic', 'race_encode_Not Specified',
       'race_encode_South American', 'admission_type_DIRECT EMER.',
       'admission_type_DIRECT OBSERVATION', 'admission_type_ELECTIVE',
       'admission_type_EU OBSERVATION', 'admission_type_EW EMER.',
       'admission_type_OBSERVATION ADMIT',
       'admission_type_SURGICAL SAME DAY ADMISSION', 'admission_type_URGENT',
       'first_careunit_Cardiac Vascular Intensive Care Unit (CVICU)',
       'first_careunit_

In [ ]:
scaler = StandardScaler()

train_normalized_cat0 = scaler.fit_transform(train_numeric_cat0)
test_normalized_cat0 = scaler.transform(test_numeric_cat0)
holdout_normalized_cat0 = scaler.transform(holdout_numeric_cat0)

train_normalized_cat1 = scaler.fit_transform(train_numeric_cat1)
test_normalized_cat1 = scaler.transform(test_numeric_cat1)
holdout_normalized_cat1 = scaler.transform(holdout_numeric_cat1)

In [ ]:
n_features = train_normalized_cat0.shape[1]

# Reshape to (n_samples, n_timepoints, n_features)
train_reshaped_cat0 = train_normalized_cat0.reshape(-1, 5, n_features)
test_reshaped_cat0 = test_normalized_cat0.reshape(-1, 5, n_features)
holdout_reshaped_cat0 = holdout_normalized_cat0.reshape(-1, 5, n_features)

train_reshaped_cat1 = train_normalized_cat1.reshape(-1, 5, n_features)
test_reshaped_cat1 = test_normalized_cat1.reshape(-1, 5, n_features)
holdout_reshaped_cat1 = holdout_normalized_cat1.reshape(-1, 5, n_features)

train_reshaped_cat1.shape, test_reshaped_cat1.shape, holdout_reshaped_cat1.shape

((4198, 5, 61), (1150, 5, 61), (614, 5, 61))

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

def build_cnn_model(input_shape):
    model = Sequential([
        Conv1D(filters=32, kernel_size=2, activation='relu', input_shape=input_shape),
        MaxPooling1D(pool_size=2),
        Conv1D(filters=64, kernel_size=2, activation='relu'),
        Flatten(),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model


In [ ]:
input_shape = (5, train_reshaped_cat0.shape[2])

model_cat0 = build_cnn_model(input_shape)
model_cat1 = build_cnn_model(input_shape)

model_cat0.summary()
model_cat1.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_6 (Conv1D)           (None, 4, 32)             3936      
                                                                 
 max_pooling1d_6 (MaxPoolin  (None, 2, 32)             0         
 g1D)                                                            
                                                                 
 conv1d_7 (Conv1D)           (None, 1, 64)             4160      
                                                                 
 flatten_3 (Flatten)         (None, 64)                0         
                                                                 
 dense_6 (Dense)             (None, 64)                4160      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                      

In [ ]:
train_targets_cat0 = train_data_cat0['los_icu']
test_targets_cat0 = test_data_cat0['los_icu']
holdout_targets_cat0 = holdout_data_cat0['los_icu']

train_targets_cat1 = train_data_cat1['los_icu']
test_targets_cat1 = test_data_cat1['los_icu']
holdout_targets_cat1 = holdout_data_cat1['los_icu']

In [ ]:
train_targets_cat0 = np.array(train_targets_cat0).reshape(-1)
test_targets_cat0 = np.array(test_labels_df_cat0).reshape(-1)

In [ ]:
train_targets_cat0 .shape

(31240,)

In [ ]:
history_cat0 = model_cat0.fit(train_reshaped_cat0, train_targets_cat0, epochs=10, validation_data=(holdout_reshaped_cat0, holdout_targets_cat0))
history_cat1 = model_cat1.fit(train_reshaped_cat1, train_targets_cat1, epochs=10, validation_data=(holdout_reshaped_cat1, holdout_targets_cat1))

In [ ]:
model_cat0.evaluate(holdout_reshaped_cat0, holdout_labels_cat0)
model_cat1.evaluate(holdout_reshaped_cat1, holdout_labels_cat1)

## **Prediction of length of stay (los_icu)**

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, MaxPooling1D

model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(5,61)),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=128, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(50, activation='relu'),
    Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_2 (Conv1D)           (None, 60, 64)            256       
                                                                 
 max_pooling1d_2 (MaxPoolin  (None, 30, 64)            0         
 g1D)                                                            
                                                                 
 conv1d_3 (Conv1D)           (None, 28, 128)           24704     
                                                                 
 max_pooling1d_3 (MaxPoolin  (None, 14, 128)           0         
 g1D)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 1792)              0         
                                                                 
 dense_2 (Dense)             (None, 50)               

In [ ]:
train_targets_cat0 = train_data_cat0['los_icu']
test_targets_cat0 = test_data_cat0['los_icu']
holdout_targets_cat0 = holdout_data_cat0['los_icu']

train_targets_cat1 = train_data_cat1['los_icu']
test_targets_cat1 = test_data_cat1['los_icu']
holdout_targets_cat1 = holdout_data_cat1['los_icu']

In [ ]:
history_cat0 = model.fit(train_reshaped_cat0, train_targets_cat0, epochs=5, validation_data = (holdout_reshaped_cat0, holdout_targets_cat0))

501/977 [==============>...............] - ETA: 6s - loss: 0.1581 - mae: 0.3371

KeyboardInterrupt: 

In [ ]:
history_cat1 = model.fit(train_reshaped_cat1, train_targets_cat1, epochs=10, validation_data = (holdout_reshaped_cat1, holdout_targets_cat1))

Epoch 1/10
656/656 [==============================] - 6s 9ms/step - loss: 0.5203 - mae: 0.6000 - val_loss: 0.5264 - val_mae: 0.6155
Epoch 2/10
656/656 [==============================] - 11s 16ms/step - loss: 0.4283 - mae: 0.5483 - val_loss: 0.5963 - val_mae: 0.6587
Epoch 3/10
656/656 [==============================] - 8s 12ms/step - loss: 0.3933 - mae: 0.5199 - val_loss: 0.5925 - val_mae: 0.6380
Epoch 4/10
656/656 [==============================] - 7s 11ms/step - loss: 0.3505 - mae: 0.4863 - val_loss: 0.6133 - val_mae: 0.6572
Epoch 5/10
656/656 [==============================] - 6s 9ms/step - loss: 0.3118 - mae: 0.4535 - val_loss: 0.6247 - val_mae: 0.6600
Epoch 6/10
656/656 [==============================] - 8s 12ms/step - loss: 0.2759 - mae: 0.4228 - val_loss: 0.6616 - val_mae: 0.6688
Epoch 7/10
656/656 [==============================] - 6s 9ms/step - loss: 0.2435 - mae: 0.3928 - val_loss: 0.6895 - val_mae: 0.6861
Epoch 8/10
656/656 [==============================] - 8s 12ms/step - lo

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

NameError: name 'history' is not defined

<Figure size 1000x500 with 0 Axes>

In [ ]:
model.evaluate(test_reshaped_cat0, test_targets_cat0)

281/281 [==============================] - 1s 4ms/step - loss: 3.8528 - mae: 1.8469


[3.852846622467041, 1.8468519449234009]

In [ ]:
model.evaluate(test_reshaped_cat1, test_targets_cat1)

180/180 [==============================] - 1s 4ms/step - loss: 2605.3245 - mae: 1.8937


[2605.324462890625, 1.893737554550171]

In [ ]:
test_predict_cat0 = model.predict(test_reshaped_cat0)
test_predict_cat0 = np.round(test_predict_cat0, 2)

281/281 [==============================] - 2s 7ms/step


In [ ]:
test_predict_cat0

array([[3.15],
       [4.16],
       [3.43],
       ...,
       [3.15],
       [3.15],
       [3.37]], dtype=float32)

In [ ]:
test_targets_cat0.values

array([2.42, 2.42, 2.42, ..., 1.2 , 1.2 , 1.2 ])

## **Prediction of time-to-event (icu_death)**

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

model = Sequential([
    Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(train_reshaped.shape[1], 1)),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
train_targets_death = train_data['icu_death']

In [ ]:
history = model.fit(train_reshaped, train_targets_death, epochs=10, batch_size=32, validation_data = (holdout,holdout_data['los_icu']))

Epoch 1/10
1625/1625 [==============================] - 13s 7ms/step - loss: 0.0364 - accuracy: 0.9868 - val_loss: 0.0049 - val_accuracy: 0.9998
Epoch 2/10
1625/1625 [==============================] - 13s 8ms/step - loss: 0.0036 - accuracy: 0.9994 - val_loss: 0.0059 - val_accuracy: 0.9991
Epoch 3/10
1625/1625 [==============================] - 13s 8ms/step - loss: 0.0017 - accuracy: 0.9995 - val_loss: 0.0169 - val_accuracy: 0.9924
Epoch 4/10
1625/1625 [==============================] - 12s 8ms/step - loss: 0.0022 - accuracy: 0.9997 - val_loss: 8.9034e-04 - val_accuracy: 0.9995
Epoch 5/10
1625/1625 [==============================] - 13s 8ms/step - loss: 0.0016 - accuracy: 0.9997 - val_loss: 0.0010 - val_accuracy: 0.9997
Epoch 6/10
1625/1625 [==============================] - 13s 8ms/step - loss: 3.5384e-05 - accuracy: 1.0000 - val_loss: 6.1431e-04 - val_accuracy: 0.9998
Epoch 7/10
1625/1625 [==============================] - 13s 8ms/step - loss: 9.1299e-06 - accuracy: 1.0000 - val_loss:

In [ ]:
test_targets_death = test_data['icu_death']
test_loss, test_accuracy = model.evaluate(test_reshaped, test_targets_death)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')

587/587 [==============================] - 2s 4ms/step - loss: 6.2128e-04 - accuracy: 0.9999
Test Loss: 0.0006212809821590781, Test Accuracy: 0.9999467730522156


In [ ]:
predictions = model.predict(test_reshaped)

587/587 [==============================] - 3s 5ms/step


In [ ]:
binary_predictions = (predictions > 0.5).astype(int)